# http://tau-data.id/unhas/

<center><img alt="" src="images/0_Cover.jpg"/></center> 

## <center><font color="blue">Modul 04: Topic Modelling</font></center>
<b><center>(C) Taufik Sutanto - 2019</center>
<center>tau-data Indonesia ~ https://tau-data.id ~ taufik@tau-data.id</center>

## <center><font color="blue">Topic Modelling</font></center>
<img alt="" src="images/PDS_logo.jpg" />

# <font color="blue">Workshop Schedule</font>

<center><img alt="" src="images/Outline.jpeg"/></center> 

In [ ]:
# Installing Modules for Google Colab
!wget https://raw.githubusercontent.com/taufikedys/UnHas/master/taudata.py
!mkdir data
!wget -P data/ https://raw.githubusercontent.com/taufikedys/UnHas/master/data/Tweets.json
!wget -P data/ https://raw.githubusercontent.com/taufikedys/UnHas/master/data/dataTweet.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/UnHas/master/data/slang.dic
!wget -P data/ https://raw.githubusercontent.com/taufikedys/UnHas/master/data/stopwords_id.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/UnHas/master/data/stopwords_en.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/UnHas/master/data/all_indo_man_tag_corpus_model.crf.tagger
!wget -P data/ https://raw.githubusercontent.com/taufikedys/UnHas/master/data/kata_dasar.txt
!pip install unidecode
!pip install pyLDAvis
!pip install textblob
!pip install sastrawi
!pip install twython
!pip install tweepy
!pip install spacy
!pip install python-crfsuite
!python -m spacy download en
!python -m spacy download xx
!python -m spacy download en_core_web_sm
import nltk
nltk.download('popular')

In [ ]:
# Importing Modules untuk Notebook ini
import warnings; warnings.simplefilter('ignore')
import taudata as tau, itertools, re, pickle, pyLDAvis, pyLDAvis.sklearn, spacy, nltk, urllib.request
import time, numpy as np, matplotlib.pyplot as plt, networkx as nx, pandas as pd, seaborn as sns 
from matplotlib.colors import ListedColormap
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tag import CRFTagger
from gensim.models import Phrases
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from nltk.corpus import stopwords
from gensim.models import Word2Vec, FastText

pyLDAvis.enable_notebook()
sns.set(style="ticks", color_codes=True)
random_state = 170

## <font color="blue">Outline Topic Modelling :</font>
* Pendahuluan Topic Modelling
* Soft Clustering (Topic Modelling): LDA dan NMF
* Visualisasi dan Interpretasi

<h3>Ketika mengolah dokumen (file dalam bentuk teks), harapan kita seperti ini:</h3>

<img alt="" src="images/4_harapan_LSA.png" style="height:99px; width:198px" />


<h3>Namun kita sudah bahas kemarin kenyataannya seperti ini:</h3>

<p><img alt="" src="images/4_kenyataan_LSA.png" style="height:183px; width:182px" /></p>

<h2 id="Topic-Modelling-1-:-Latent-Dirichlet-Allocation">Topic Modelling 1 : Latent Dirichlet Allocation</h2>

<p><img alt="" src="images/4_Document_to_Topics.png" style="height: 300px ; width: 582px" /></p>

<p><strong><big>Tapi bukan seperti klasifikasi dan bukan berarti kata-kata Sport, Technology, dan Entertainment dominan di kategori-kategori tersebut. Topic modelling lebih ke soft-clustering, dimana suatu dokumen dimasukkan ke dalam beberapa cluster (topic) sekaligus. Adapun nama &quot;topic/cluster&quot;-nya di interpretasi dari kata-kata yang ada didalamnya.</big></strong></p>


<p><img alt="" src="images/4_LDA vs LDA.JPG" style="height:400px; width:606px" /></p>
[<a href="http://chdoig.github.io/pytexas2015-topic-modeling/" target="_blank">Sumber gambar ini dan beberapa gambar selanjutnya</a>]</p>

<p><img alt="" src="images/4_definisi topic model.JPG" style="height:350px; width:809px" />


<p><img alt="" src="images/4_inti_LDA.JPG" style="height:500px; width:785px" /></p>
Penjelasan intuitif yang baik: https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d 

<h3>Evaluasi LDA?</h3>

<p><img alt="" src="images/4_Evaluasi_LDA.jpg" style="height:400px; width:888px" /></p>
[Cara lain: http://mimno.infosci.cornell.edu/slides/details.pdf]

<p><img alt="" src="images/4_LDA Pipeline.JPG" style="height:300px; width:663px" /></p>
* Modifikasi dapat dilakukan dengan "pos tags"

In [ ]:
# stopId, lemmaId = tau.LoadStopWords(lang='id') 
print('Loading Data  ... ')
T2 = tau.loadTweets(file='data/Tweets.json')
D = [t['full_text'] for t in T2] # Tweet hasil crawling

In [ ]:
print('Loading Slang, stopwords, dan fungsi lemma ... ')

slangFixId = tau.loadCorpus(file = 'data/slang.dic', sep=':')
stopId, lemmaId = tau.LoadStopWords(lang='id') # kita akan menggunakan stopwords dan Lemma

In [ ]:
print('Cleaning Data ... ')
data = []
for i, d in tqdm(enumerate(D)):
    doc = tau.cleanText(d, fix=slangFixId, lemma=lemmaId, stops = set(stopId))
    data.append(doc)
print(data[:3])

In [ ]:
# Kita mulai dengan membuat VSM-nya
# kita gunakan perintah yang ada di Segmen sebelumnya 
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer()

tf = tf_vectorizer.fit_transform(data)
tf_terms = tf_vectorizer.get_feature_names()
# Mengapa tf bukan tfidf?
# Blei, D. M., Ng, A. Y., & Jordan, M. I. (2003). Latent dirichlet allocation. Journal of machine Learning research, 3(Jan), 993-1022.
# Bisa di tamahkan dengan Frequency filtering "Max_df" dan "Min_df"

tf.shape

In [ ]:
# Dilanjutkan dengan membentuk model LDA-nya
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_topics = 3
lda = LDA(n_components=n_topics, learning_method='batch', random_state=0).fit(tf)   
lda

In [ ]:
# Melihat Topik-topiknya
vsm_topics = lda.transform(tf)
print(vsm_topics.shape)
vsm_topics
# Ukuran kolom = #Topics ==> Dimension Reduction

In [ ]:
# Seandainya diasumsikan 1 dokumen hanya 1 topic dengan nilai skor topic terbesar
doc_topic =  [a.argmax()+1 for a in tqdm(vsm_topics)] # topic of docs
doc_topic[:10]

In [ ]:
# mari kita plot
plot = sns.countplot(doc_topic)

In [ ]:
# Mari kita coba maknai masing-masing topic ini
Top_Words = 15
print('Printing top {0} Topics, with top {1} Words:'.format(n_topics, Top_Words))
tau.print_Topics(lda, tf_terms, n_topics, Top_Words)

In [ ]:
# %matplotlib inline
# Mari kita Plot, supaya lebih jelas
# Catatan, bergantung dari laptop yang digunakan, image terkadang cukup lama untuk muncul.
import pyLDAvis, pyLDAvis.sklearn; pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)  

# Bagaimana jika kita ingin menggunakan semi-supervised (guided) LDA?
https://medium.freecodecamp.org/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164 

# Bagaimana melakukan Post-Tag sebelum Topic Modelling?

In [ ]:
from spacy.lang.id import Indonesian
from nltk.tag import CRFTagger
nlp_id = Indonesian()  # Language Model
ct = CRFTagger()
ct.set_model_file('data/all_indo_man_tag_corpus_model.crf.tagger')

def NLPfilter(t, filters):
    tokens = nlp_id(t)
    tokens = [str(k) for k in tokens if len(k)>2]
    hasil = ct.tag_sents([tokens])
    return [k[0] for k in hasil[0] if k[1] in filters]

In [ ]:
filters = set(['NN', 'NNP', 'NNS', 'NNPS', 'JJ'])

for i, d in tqdm(enumerate(data)):
    data[i] = NLPfilter(d,filters)

' '.join(data[1])

In [ ]:
print(data[:3])

# Evaluasi: Bagaimana mendapatkan parameter Optimal Topic Modelling?

**Beberapa catatan penting**:
1. Hati-hati Struktur Data, untuk melakukan evaluasi Topic Modelling struktur data yang digunakan mirip dengan Word Embedding.
2. Kita akan melakukan cross-validasi dan N-Gram
3. Ada berbagai metric evaluasi https://radimrehurek.com/gensim/models/coherencemodel.html
4. Referensi paper: http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf

In [ ]:
bigram_t = Phrases(data, min_count=10)
trigram_t = Phrases(bigram_t[data])
for idx, d in enumerate(data):
    for token in bigram_t[d]:
        if '_' in token:# Token is a bigram, add to document.
            data[idx].append(token)
    for token in trigram_t[d]:
        if '_' in token:# Token is a bigram, add to document.
            data[idx].append(token)

# Create a dictionary representation of the documents.
# Remove rare & common tokens
dictionary_t = Dictionary(data)
dictionary_t.filter_extremes(no_below=10, no_above=0.2)
#Create dictionary and corpus required for Topic Modeling
corpus_t = [dictionary_t.doc2bow(doc) for doc in data]
print('Number of unique tokens: %d' % len(dictionary_t))
print('Number of documents: %d' % len(corpus_t))
print(corpus_t[:1])


In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, coherence='c_v', start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, limit, step)):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Caution cell berikut ini membutuhkan waktu yang cukup signifikan untuk selesai, karena selain LDA *computationally expensive* loopingnya juga melakukan Cross-validasi di setiap jumlah topik *k*

In [ ]:
start, step, limit = 2, 1, 10 # Ganti dengan berapa banyak Topic yang ingin di hitung/explore
coh_t, kCV = [], 3 # hati-hati sangat lambat karena cross validation pada metode yang memang tidak efisien (LDA)

for i in tqdm(range(kCV)):
    model_list, c = compute_coherence_values(dictionary=dictionary_t, corpus=corpus_t, texts=data, start=start, limit=limit, step=step)
    coh_t.append(c)
    
coherence_t = np.mean(np.array(coh_t), axis=0)
'Done'

In [ ]:
# Show graph
x = range(start, limit, step)
plt.figure(figsize=(12,10))
for c in coh_t:
    plt.plot(x, c, '--', color = 'lawngreen', linewidth = 2)
plt.plot(x, coherence_t, '-', color = 'black', linewidth = 5)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Referensi Pilihan:

* perhitungan Manual Topic Modelling LDA: http://brooksandrew.github.io/simpleblog/articles/latent-dirichlet-allocation-under-the-hood/
* http://mimno.infosci.cornell.edu/slides/details.pdf
* https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/
* http://www.umiacs.umd.edu/~jbg/docs/nips2009-rtl.pdf
* http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html
* Penjelasan intuitif yang baik: https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d 
* inconjunction dengan interactive program berikut: https://lettier.com/projects/lda-topic-modeling/ 

<p>&nbsp;</p>

<h3 id="Non-Negative-Matrix-Decomposition-(NMF)">Non-Negative Matrix Decomposition (NMF)</h3>

<p><img alt="" src="images/4_NMF.jpg" style="height: 349px; width: 600px;" /> [image source: <a href="https://www.slideshare.net/SebastianRuder/dynamic-topic-modeling-via-nonnegative-matrix-factorization-dr-derek-greene]">https://www.slideshare.net/SebastianRuder/dynamic-topic-modeling-via-nonnegative-matrix-factorization-dr-derek-greene]</a></p>

<p>&nbsp;</p>


<h3 id="Tujuan-NMF:">Tujuan NMF:</h3>

<p><img alt="" src="images/4_NMF_Goal.JPG" style="height: 363px; width: 600px;" /></p>


In [ ]:
# Langsung Aplikasi-nya
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

data = []
for i, d in tqdm(enumerate(D)):
    doc = tau.cleanText(d, fix=slangFixId, lemma=lemmaId, stops = set(stopId))
    data.append(doc)


tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(data)
nmf_model = NMF(n_components = 3, random_state=0)
nmf = nmf_model.fit(tfidf)

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
tau.print_Topics(nmf, tfidf_feature_names, n_topics, Top_Words)

In [ ]:
# Sama seperti LDA kita bisa melihat distribusi topic setiap dokumen
vsm_topics = nmf.transform(tfidf)
vsm_topics[:10]

In [ ]:
# Seandainya diasumsikan 1 dokumen hanya 1 topic dengan nilai skor topic terbesar
doc_topic =  [a.argmax()+1 for a in tqdm(vsm_topics)] # topic of docs
print('In total there are {0} major topics, distributed as follows'.format(len(set(doc_topic))))
sns.countplot(doc_topic)
plt.show()

In [ ]:
# Perkalian Matrix-nya (Aljabar Linear)
W = nmf_model.fit_transform(tfidf)
H = nmf_model.components_
terms = tfidf_vectorizer.get_feature_names()
print(W.shape, H.shape, len(terms))
#print(terms[:5])

# Latihan :

* Load data data tweet dengan isu berbeda, "misal" **revitalisasi monas, banjir, atau isu terkini lainnya**
* Lakukan preprocessing (termasuk lemma) dan pos tag (ambil hanya noun saja)
* Bandingkan hasil topic dari LDA, dan NMF dari data tersebut.
* Apakah hasilnya sudah baik?
* Buat visualisasi pyLDAvis-nya dan analisa lebih lanjut.


# <center><font color="blue"> End of Module 04

<hr />
<p><img alt="" src="images/1_meme.jpg" /></p>